In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import s3fs
import os

Open gauge parquet file to get the corresponding pair of model and gauge

In [ ]:
df_a = pd.read_parquet('/Users/yubinbaaniya/Documents/WORLD BIAS/saber workdir/gauge_table_2nd_iteration_deDuplicated.parquet')

Open monthly model fdc zarr file

In [ ]:
ds_s = xr.open_zarr('/Users/yubinbaaniya/Documents/WORLD BIAS/saber workdir/simulated_monthly_fdc_exceedence.zarr')

OPen monthly gauge fdc zarr file

In [ ]:
ds_o = xr.open_zarr('/Users/yubinbaaniya/Documents/WORLD BIAS/saber workdir/oberved_monthly_fdc.zarr')

Calculates a SFDC for each pairs and also correct sfdc

In [ ]:
def adjust_scalar_fdc(scalar_fdc):
    """
    Adjust scalar values and fill missing p_exceed values
    
    Parameters:
    scalar_fdc: DataFrame with scalar values indexed by p_exceed
    
    Returns:
    DataFrame with adjusted scalar values for full p_exceed range (0-100)
    """
    # Create a copy of the original DataFrame to avoid altering it
    adjusted_fdc = scalar_fdc.copy()
    
    # Step 1: Set scalars to 1 if they are:
    # - less than 0.001
    # - inf (from division by zero)
    # - nan (from invalid division)
    adjusted_fdc['scalars'] = adjusted_fdc['scalars'].apply(
        lambda x: 1 if (x < 0.001) or np.isinf(x) or np.isnan(x) else x
    )
    
    # Step 2: Fill missing p_exceed values down to 0 with scalars set to 1
    full_range = pd.Index(range(0, 101))
    missing_p_exceed = full_range.difference(adjusted_fdc.index)
    
    if len(missing_p_exceed) > 0:  # If there are any missing values
        missing_rows = pd.DataFrame({'scalars': 1}, index=missing_p_exceed)
        adjusted_fdc = pd.concat([adjusted_fdc, missing_rows]).sort_index(ascending=False)
    
    return adjusted_fdc

def extract_and_process_fdc_values(ds_s, ds_o, df_a, index=None):
    """
    Extract FDC values and calculate adjusted scalars for specified index(es)
    
    Parameters:
    ds_s: xarray Dataset - source zarr with rivid dimension
    ds_o: xarray Dataset - source zarr with gauge dimension
    df_a: DataFrame - mapping between model_id and gauge_id
    index: int or None - if provided, process only that index; if None, process all rows
    
    Returns:
    DataFrame with rivid, month, p_exceed, and scalar values
    """
    final_rows = []
    
    # Determine which rows to process
    if index is not None:
        # Process single index
        rows_to_process = [df_a.iloc[index]]
    else:
        # Process all rows
        rows_to_process = [row for _, row in df_a.iterrows()]
    
    # Process each row
    for row in rows_to_process:
        rivid = row['model_id']
        gauge_id = row['gauge_id']
        
        try:
            # Get values from ds_s (model data)
            model_fdc = ds_s.fdc.sel(rivid=rivid).values
            
            # Get values from ds_o (gauge data)
            gauge_fdc = ds_o.fdc.sel(gauge=gauge_id).values
            
            # Process each month separately
            for month in range(1, 13):
                # Create monthly DataFrame
                month_df = pd.DataFrame({
                    'rivid': rivid,
                    'month': month,
                    'p_exceed': ds_s.p_exceed.values,
                    'scalar': model_fdc[:, month-1] / gauge_fdc[:, month-1]
                })
                
                # Sort by p_exceed descending (100 to 0) for adjustment function
                month_df = month_df.sort_values('p_exceed', ascending=False)
                
                # Create DataFrame with scalar values indexed by p_exceed for adjustment
                scalar_df = pd.DataFrame({'scalars': month_df['scalar'].values}, 
                                       index=month_df['p_exceed'])
                
                # Adjust scalar values using the adjustment function
                adjusted_df = adjust_scalar_fdc(scalar_df)
                
                # Replace original scalar values and ensure all p_exceed values are present
                month_df = pd.DataFrame({
                    'rivid': rivid,
                    'month': month,
                    'p_exceed': adjusted_df.index,
                    'scalar': adjusted_df['scalars'].values
                })
                
                final_rows.append(month_df)
        
        except KeyError as e:
            print(f"Warning: Could not find data for rivid {rivid} or gauge_id {gauge_id}")
            continue
    
    # Combine all data
    if final_rows:
        result_df = pd.concat(final_rows, ignore_index=True)
        # Ensure columns are in desired order
        result_df = result_df[['rivid', 'month', 'p_exceed', 'scalar']]
        return result_df
    else:
        return None


# For single index
#df_result_single = extract_and_process_fdc_values(ds_s, ds_o, df_a, index=0)

melted_df = extract_and_process_fdc_values(ds_s, ds_o, df_a)

Create a zarr file for that sfdc also

In [ ]:
#ZARR functions
def create_xarray_zarr(melted_df):
    """
    Convert melted DataFrame to xarray Dataset and save as Zarr
    
    Parameters:
    melted_df (pd.DataFrame): Melted DataFrame with columns 'rivid', 'Month', 'p_exceed', 'fdc'
    """
    # Ensure proper data types
    melted_df = melted_df.copy()
    melted_df['Month'] = melted_df['Month'].astype(int)
    melted_df['p_exceed'] = melted_df['p_exceed'].astype(int)
    
    # Sort values to ensure consistent ordering
    melted_df = melted_df.sort_values(['rivid', 'p_exceed', 'Month'])
    
    # Get unique values for dimensions
    gauges = sorted(melted_df['rivid'].unique())
    p_exceed_values = sorted(melted_df['p_exceed'].unique())
    months = sorted(melted_df['Month'].unique())
    
    # Create 3D array with proper shape
    shape = (len(gauges), len(p_exceed_values), len(months))
    data = np.full(shape, np.nan)
    
    # Create lookup dictionaries for faster indexing
    gauge_idx = {g: i for i, g in enumerate(gauges)}
    p_idx = {p: i for i, p in enumerate(p_exceed_values)}
    month_idx = {m: i for i, m in enumerate(months)}
    
    # Fill the 3D array
    for _, row in melted_df.iterrows():
        i = gauge_idx[row['rivid']]
        j = p_idx[row['p_exceed']]
        k = month_idx[row['Month']]
        data[i, j, k] = row['scalar']
    
    # Create xarray Dataset
    ds = xr.Dataset(
        {
            'fdc': (['rivid', 'p_exceed', 'month'], data)
        },
        coords={
            'rivid': gauges,
            'p_exceed': p_exceed_values,
            'month': months
        }
    )
    
    # Chunk the dataset - adjust chunk sizes based on your needs
    ds = ds.chunk({
        'rivid': min(50, len(gauges)),
        'p_exceed': min(101, len(p_exceed_values)),
        'month': min(12, len(months))
    })
    
    return ds

def save_to_zarr(ds, filename='/Users/yubinbaaniya/Documents/WORLD BIAS/saber workdir/simulated_sfdc.zarr'):  #the file path is hardcoded here
    """
    Save xarray Dataset to Zarr format and return the absolute path
    
    Parameters:
    ds (xarray.Dataset): Dataset to save
    filename (str): Output filename with path
    
    Returns:
    str: Absolute path to the saved Zarr file
    """
    # Convert to absolute path
    abs_path = os.path.abspath(filename)
    
    # Save to Zarr format without compression
    ds.to_zarr(abs_path, mode='w')
    
    print(f"Zarr file saved to: {abs_path}")
    return abs_path


#Function Call to make a zarr file
ds = create_xarray_zarr(melted_df)
zarr_path = save_to_zarr(ds)